In [21]:
import os
import sys
import pickle
import random
import numpy as np
import pandas as pd
sys.path.append('../src/')
from configs import *
from utils_optimization import *
from tqdm.notebook import tqdm

from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
rcParams.update(fig_params)
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data and models

In [22]:
## Featurizer
featurizer = Featurizer(data_folder=os.path.join('..', 'data'))

## Model
with open(os.path.join('..', 'models', 'efficient_model.pickle'), 'rb') as handle:
    models = pickle.load(handle)
best_iteration = models['best_iteration']
n_models = 1 #len(models['models_ensemble'].boosters)

## Train dataset
train, features = load_dataset()

## Selected features
selected_features = ['CTDC_charge.G1',
                     'CTDD_hydrophobicity_ARGP820101.1.residue50',
                     'SOCNumber_gGrantham.lag1',
                     'DDE_LA',
                     'Geary_ANDN920101.lag2',
                     'DDE_SP',
                     'NetC',
                     'ASDC_VG',
                     'CTDC_solventaccess.G3',
                     'APAAC_Pc1.V']

## Optimization loop

In [23]:
TARGET = 'RNSYRKSPSRRNR'
len_gnome = len(TARGET)
max_iter = 50
size_population = 100
genes = '''ARNDCEQGHILKMFPSTWYV'''

# Training features for anomaly detection
feat_train = train[selected_features]
clf_anomaly = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination=0.1)
clf_anomaly.fit(feat_train)

# Sequence alignment settings
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.target_internal_open_gap_score = -20
aligner.query_internal_open_gap_score = -20

generation = 1
population = []

optimization_info = {'Pop_size': size_population, 'max_iter': max_iter}

params_fit = {'obj': my_fitness, 'target_ligand': TARGET, 'aligner': aligner, 'model': models, 'clf_anomaly': clf_anomaly, 
             'featurizer': featurizer, 'best_iteration': best_iteration, 'n_models': n_models}

features_TARGET = featurizer.compute_features(TARGET)
print(f"Penetrability of target: {pred_penetration(features_TARGET, models['models_refit'], best_iteration, n_models)}")
print(f"Anomaly of target: {anomaly_score(features_TARGET, clf_anomaly)}")

# Create initial population
max_diff_pct = 50  # set the percentage of different letters, user can change this value
for _ in range(size_population):
    gnome = Individual.create_gnome(len_gnome, genes, TARGET, max_diff_pct)
    population.append(Individual(gnome, params_fit))

# Define stopping criteria thresholds
penetration_threshold = 0.95
fitness_threshold = 0.1
anomaly_threshold = 1.3

best_sequences = {}

while generation <= max_iter:
    # Sort the population in increasing order of fitness score
    population = sorted(population, key=lambda x: x.fitness)

    for individual in population:
        if individual.num_diff_residues not in best_sequences:
            best_sequences[individual.num_diff_residues] = individual
        else:
            if individual.fitness < best_sequences[individual.num_diff_residues].fitness:
                best_sequences[individual.num_diff_residues] = individual

    # If the best solution in the population meets the stopping criteria, break the loop
    best_solution = population[0]
    features_best = featurizer.compute_features(''.join(best_solution.chromosome))
    penetration_score = pred_penetration(features_best, models['models_refit'], best_iteration, n_models)
    anomaly_score_value = anomaly_score(features_best, clf_anomaly)[0]

    # Including the condition for num_diff_residues
    if (penetration_score > penetration_threshold and anomaly_score_value < anomaly_threshold and best_solution.num_diff_residues <= 1):
        break
    # Including the condition for num_diff_residues
    if (best_solution.num_diff_residues == 0):
        break
    # Generate new offsprings for new generation
    new_generation = []

    # Perform Elitism, that mean 10% of fittest population
    # goes to the next generation
    s = int((10*size_population) / 100)
    new_generation.extend(population[:s])

    # From 50% of fittest population, Individuals
    # will mate to produce offspring
    s = int((90*size_population) / 100)
    for _ in range(s):
        parent1 = random.choice(population[:50])
        parent2 = random.choice(population[:50])
        child = parent1.mate(parent2, genes)
        new_generation.append(child)

    population = new_generation

    print(f"Generation: {generation} -- "
          f"Best solution: {''.join(population[0].chromosome)} -- "
          f"Best fitness: {population[0].fitness:.5f} -- "
          f"Prob_penetration: {penetration_score:.2f} -- "
          f"Anomaly score: {anomaly_score_value:.2f} -- "
          f"Different letters from target: {population[0].num_diff_residues}"
          )

    generation += 1

print(f"Generation: {generation} -- "
      f"Best solution: {''.join(population[0].chromosome)} -- "
      f"Best fitness: {population[0].fitness:.5f} -- "
      f"Prob_penetration: {penetration_score:.2f} -- "
      f"Anomaly score: {anomaly_score_value:.2f} -- "
      f"Different letters from target: {population[0].num_diff_residues}"
      )

features_opt = featurizer.compute_features(''.join(population[0].chromosome))
distance = compute_distance(aligner, TARGET, ''.join(population[0].chromosome))

print(f"Generation: {generation} -- "
      f"Best solution: {''.join(population[0].chromosome)} -- "
      f"Best fitness: {population[0].fitness} -- "
      f"Different letters from target: {population[0].num_diff_residues}"
      )

# Print the best sequence for each number of different letters
for num_diff_residues, individual in sorted(best_sequences.items()):
    print(f"Best solution with {num_diff_residues} different letters from target: "
          f"{''.join(individual.chromosome)} -- "
          f"Prob_penetration: {pred_penetration(featurizer.compute_features(''.join(individual.chromosome)), models['models_refit'], best_iteration, n_models):.2f} -- "
          f"Anomaly score: {anomaly_score(featurizer.compute_features(''.join(individual.chromosome)), clf_anomaly)[0]:.2f}"
          )

Penetrability of target: 0.9366376658161918
Anomaly of target: [0.99796328]
Generation: 1 -- Best solution: RNSYRKSPSRRVH -- Best fitness: 0.20588 -- Prob_penetration: 0.98 -- Anomaly score: 1.04 -- Different letters from target: 2
Generation: 2 -- Best solution: RNSYQKPPQRRNR -- Best fitness: 0.19118 -- Prob_penetration: 0.88 -- Anomaly score: 0.99 -- Different letters from target: 3
Generation: 3 -- Best solution: RNSYRKSPSRRYR -- Best fitness: 0.11765 -- Prob_penetration: 0.96 -- Anomaly score: 0.99 -- Different letters from target: 1
Generation: 3 -- Best solution: RNSYRKSPSRRYR -- Best fitness: 0.11764705882352944 -- Different letters from target: 1
Best solution with 1 different letters from target: RNSYRKSPSRRYR -- Prob_penetration: 0.96 -- Anomaly score: 0.99
Best solution with 2 different letters from target: RDSYRKSPSARNR -- Prob_penetration: 0.92 -- Anomaly score: 1.26
Best solution with 3 different letters from target: RNSYQKPPQRRNR -- Prob_penetration: 0.88 -- Anomaly scor